In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split


In [ ]:
data = pd.read_csv("/Users/keshavsaraogi/data/wikihowAll.csv")

print(data.columns)

In [ ]:
print(data.head())
print(data.info())
print(data.describe())

In [ ]:
# Select relevant columns

df = data[['title', 'headline', 'text']].dropna()
print(df)

In [ ]:
train_texts, val_texts, train_summaries, val_summaries = train_test_split(
    df['text'].tolist(), df['headline'].tolist(), test_size=0.1, random_state=42
)

In [ ]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)